## Simulating data to benchmark FC algorithms

In [1]:
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import savemat, loadmat
from scipy.stats import linregress
import os
import networkx as nx

In [2]:
# defining parameters

# general
identifier='sim09_2022-04-19_r1'
# standardised format 
    #'sim##_YYYY_MM_DD_r#'; 
        #'sim##' is incremented for a different network instantiation, 
        #'r#' is incrememnted for different simulations of same network instantiation

# topology
top = 'rand' # 'rand' (random) or 'sfree' (scale-free)
N = 1000 # neurons
#p_connect = 0.005 if top == 'rand' else print('') # (for 'rand') synnaptic connection probability
p_connect = 0.005 if top == 'rand' else print('') # (for 'rand') synnaptic connection probability
sfg_seed = 42 # (for 'sfree') seed for generating scale free graph

# dynamics
sig = 0.08 # independent noise
threshold = 1 
reset = 0.9 # reset value after a spike initiation
refractory = 2*ms # absolute refracotry period
rest_pot_I = 0.9 # steady state value for membrane dynamics ('resting potential')
mem_tau = 10*ms # membrane time constant
syn_w = 0.025 # synaptic weight
syn_dt = 1*ms # synaptic delay

# time
run_for = 1800 # 1800 ('30 minutes')
imaging_rate = 30 # for binning data

In [3]:
t_stamp =  str(np.datetime64('now')).replace('-','').replace('T','_').replace(':', '') # for saving

save_path = os.getcwd() + '/data' # path to general ('ani' and 'sim') data folder
fig_path = f'{save_path}/{identifier}/sim_figures' # path to general ('ani' and 'sim') data folder

os.makedirs(f'{save_path}/{identifier}/', exist_ok=True)
os.makedirs(fig_path, exist_ok=True)

In [ ]:
# define neurons and connectivity

start_scope()

eqs = '''
dv/dt = (I-v)/tau + sig*xi*tau**-0.5: 1
I : 1
tau : second
'''

G = NeuronGroup(N, eqs, threshold=f'v>{threshold}', reset=f'v = {reset}', refractory=refractory, method='euler')
G.I = rest_pot_I
G.tau = mem_tau #originally 10
S = Synapses(G, G, on_pre=f'v_post += {syn_w}', delay=syn_dt)

if top == 'rand':
    
    # connecting randomly
    S.connect(condition='i!=j', p=p_connect)

elif top == 'sfree':
    
    # getting scale free network
    sf_graph = nx.scale_free_graph(N, seed=sfg_seed)
    adj = nx.adjacency_matrix(sf_graph).toarray()
    np.fill_diagonal(adj, 0) # this will somewhat alter scale-free topology
    adj_bin = np.copy(adj) 
    adj_bin[adj_bin>=1]=1
    syns = np.nonzero(adj_bin) # sparse matrix format expected by brian
    
    # connecting scale-freely
    S.connect(i=syns[1],j=syns[0]) # hmmm - here think which are the incoming and which the outgoing

# run experiment
spikemon = SpikeMonitor(G)

run(run_for*second, report='stdout', report_period=10*second)

Starting simulation at t=0. s for a duration of 1.8 ks
11.1941 s (0%) simulated in 10s, estimated 26m 38s remaining.
22.7993 s (1%) simulated in 20s, estimated 25m 59s remaining.
33.1861 s (1%) simulated in 30s, estimated 26m 37s remaining.
44.2355 s (2%) simulated in 40s, estimated 26m 28s remaining.
54.0285 s (3%) simulated in 50s, estimated 26m 56s remaining.
62.0572 s (3%) simulated in 1m 0s, estimated 28m 0s remaining.
72.6658 s (4%) simulated in 1m 10s, estimated 27m 44s remaining.
84.4884 s (4%) simulated in 1m 20s, estimated 27m 4s remaining.
96.3569 s (5%) simulated in 1m 30s, estimated 26m 31s remaining.
108.2248 s (6%) simulated in 1m 40s, estimated 26m 3s remaining.
119.965 s (6%) simulated in 1m 50s, estimated 25m 41s remaining.
131.4616 s (7%) simulated in 2m 0s, estimated 25m 23s remaining.
140.8805 s (7%) simulated in 2m 10s, estimated 25m 31s remaining.
151.5321 s (8%) simulated in 2m 20s, estimated 25m 23s remaining.
162.7155 s (9%) simulated in 2m 30s, estimated 25m 

In [ ]:
# visualise connectivity
plt.figure(figsize=(10,10))
plt.scatter(S.i, S.j, s=1, c='k')
plt.ylabel('post-synaptic neuron')
plt.xlabel('pre-synaptic neuron')
plt.savefig(f'{fig_path}/{t_stamp}_ground_truth.png')

In [ ]:
# computing degrees and fit on linear-linear plot
degrees=[]
for k in range(1000):
    degrees.append(np.sum(S.i==k) + np.sum(S.j==k))
    
x_loglog = np.log10(np.arange(1,len(degrees)+1))
y_loglog = np.log10(np.flip(np.sort(degrees)))#+0.000001)
coeffs = linregress(x_loglog, y_loglog) # linear fit

In [ ]:

plt.figure(figsize=(10,10))
plt.plot(x_loglog, y_loglog, label='log-log degrees')
plt.scatter(x_loglog, y_loglog, label='log-log degrees', s=5)
plt.plot(x_loglog, coeffs[0]*x_loglog + coeffs[1], '--',c='grey', label='linear fit')
plt.xlabel('log(degree)')
plt.ylabel('log(number of notes)')
plt.legend()
plt.savefig(f'{fig_path}/{t_stamp}_loglog_degree.png')

In [ ]:
# binning at a particular imaging rate (binarised raster) and getting correlation matrix
n_neurons = N
all_hist = np.zeros((n_neurons, run_for*imaging_rate))
for n in range(n_neurons):
    all_hist[n,:] = np.histogram(spikemon.t[spikemon.i==n], bins=np.arange(0,run_for+0.0001,1/imaging_rate))[0]
    #all_hist.append(n_hist[0])

corr_mat = np.corrcoef(all_hist)
#adding zeros to diagonal (no self-connections) - for better visualisation
np.fill_diagonal(corr_mat, 0)

In [ ]:
# exporting simulated data to compare different FC inference techniques

sim_data = {'syn_i': np.copy(S.i),
            'syn_j': np.copy(S.j),
           'st': np.copy(spikemon.t),
           'si': np.copy(spikemon.i),
           'imaging_rate': imaging_rate,
           'sts_binned': all_hist,
           'corr_mat': corr_mat}

np.save(f'{save_path}/{identifier}/{identifier}.npy', sim_data)
savemat(f'{save_path}/{identifier}/{identifier}.mat', sim_data)

## Plotting everything

In [ ]:
# plot spike times for each neuron (true raster)
xlim = [0, 1000]
plt.figure(figsize=(20, 8), dpi=1000)
plt.scatter(spikemon.t[spikemon.i<1000]/ms-1000, spikemon.i[spikemon.i<1000], s=0.1, c='k') # t is spike time, i is neuron id
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index')
plt.title(f'Spiking activity ({(xlim[1]-xlim[0])/1e3} seconds)')
plt.xlim(xlim)
plt.savefig(f'{fig_path}/{t_stamp}_spiking_data.png')

In [ ]:
all_hist_zscore = (all_hist - np.mean(all_hist,1)[:,np.newaxis])/np.std(all_hist,1)[:,np.newaxis]

In [ ]:
_, axs = plt.subplots(2, 1, figsize=(20,8), dpi = 200, gridspec_kw={'height_ratios': [3, 1]})

axs[0].imshow(all_hist_zscore[:,:3000], cmap='binary', interpolation='nearest', aspect='auto', vmin=0, vmax=5) # log colorscale
axs[0].xaxis.set_visible(False)
axs[0].set_ylabel('roi id')


axs[1].plot(np.mean(all_hist_zscore[:,:3000], axis=0), label='avg')
axs[1].set_xlim((0, all_hist_zscore[:,:3000].shape[1]))
axs[1].set_ylabel('avg.')
axs[1].set_xlabel('time (stamps)')

plt.savefig(f'{fig_path}/{t_stamp}_binned_30.png')

In [ ]:
# plot histogram at a particular imaging rate
plt.figure(figsize=(10, 4), dpi=1000)
plt.imshow(all_hist[:,:5000], vmax=5, cmap='Greys')
plt.savefig(f'{fig_path}/{t_stamp}_binned_30.png')
plt.axis('off')

### Connectivity inference using correlation matrix

In [ ]:
# THIS IS WITHOUT BINARISING but with binning + adding zeros on diagonal
#plotting
plt.figure(figsize=(10, 4), dpi=1000)
plt.imshow(corr_mat, cmap='binary')
plt.colorbar()
plt.scatter(S.i, S.j, s=0.01, facecolors=None, edgecolors='C3', marker='X')
plt.savefig(f'{fig_path}/{t_stamp}_emp_vs_gt_directed.png')

In [ ]:
plt.figure(figsize=(10, 4), dpi=1000)
plt.imshow(corr_mat, cmap='binary')
plt.scatter(S.i, S.j, s=0.01, facecolors=None, edgecolors='C3', marker='X')
plt.scatter(S.j, S.i, s=0.01, facecolors=None, edgecolors='C3', marker='X')
plt.savefig(f'{fig_path}/{t_stamp}_emp_vs_gt_undirected.png')